In [1]:
using JuMP, Ipopt
using LinearAlgebra
using Combinatorics: combinations
using ForwardDiff: gradient, hessian
using Random
using Plots

In [2]:
TwoBundles = collect(combinations(1:4, 2))
ThrBundles = collect(combinations(1:4, 3))

TwoOpt = [3, 4]
ThrOpt = [1, 2, 3]

setdiff!(TwoBundles, [TwoOpt])
setdiff!(ThrBundles, [ThrOpt])

display(TwoBundles)
display(ThrBundles)

5-element Vector{Vector{Int64}}:
 [1, 2]
 [1, 3]
 [1, 4]
 [2, 3]
 [2, 4]

3-element Vector{Vector{Int64}}:
 [1, 2, 4]
 [1, 3, 4]
 [2, 3, 4]

In [92]:
mod = Model(Ipopt.Optimizer)
set_silent(mod)

@variable(mod, t[1:4] ≥ 0)
@variable(mod, 0 ≤ f[1:4] ≤ 1)

@constraint(mod, Sortedness[j in 1:3], t[j] ≤ t[j+1])

ε = 0.001

h = 2
@NLconstraint(mod, TwoOptCon[l in 1:length(TwoBundles)],
    sum(t[TwoOpt[i]] * f[TwoOpt[i]] * prod(1 - f[TwoOpt[j]] for j in i+1:h) for i in 1:h-1) + t[TwoOpt[end]] * f[TwoOpt[end]] - ε ≥
    sum(t[TwoBundles[l][i]] * f[TwoBundles[l][i]] * prod(1 - f[TwoBundles[l][j]] for j in i+1:h) for i in 1:h-1) + t[TwoBundles[l][end]] * f[TwoBundles[l][end]]
)
    
h = 3
@NLconstraint(mod, ThrOptCon[l in 1:length(ThrBundles)],
    sum(t[ThrOpt[i]] * f[ThrOpt[i]] * prod(1 - f[ThrOpt[j]] for j in i+1:h) for i in 1:h-1) + t[ThrOpt[end]] * f[ThrOpt[end]] - ε ≥
    sum(t[ThrBundles[l][i]] * f[ThrBundles[l][i]] * prod(1 - f[ThrBundles[l][j]] for j in i+1:h) for i in 1:h-1) + t[ThrBundles[l][end]] * f[ThrBundles[l][end]]
)

optimize!(mod)
termination_status(mod)

LOCALLY_INFEASIBLE::TerminationStatusCode = 5

In [69]:
primal_feasibility_report(mod)

Dict{Any, Float64} with 1 entry:
  ((t[1] * f[1] * ((1.0 - f[3]) * (1.0 - f[4])) + t[3] * f[3] * … => 3.42728e-10

In [64]:
value.(t) |> println
value.(f) |> println

[2.9962542114803625, 4.315146043300834, 5.334970409723789, 9.707365021388634]
[0.8782556903051538, 0.3057100995525, 0.1903886995899347, 0.06853687639185227]


In [3]:
t = randexp(5) |> sort
f = rand(5)

function valuation(x; t=t, f=f)
    m = length(x)
    return sum(x[i] * t[i] * f[i] * prod(1 - f[j]*x[j] for j in i+1:m) for i in 1:m-1) + x[m] * t[m] * f[m]
end

hessian(valuation, [0, 1, 0, 1, 1]) |> eigen

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
5-element Vector{Float64}:
 -1.5708912927995462
 -1.8891607273317712e-6
  7.640623629540836e-7
  2.0243612650335747e-6
  1.57089039353665
vectors:
5×5 Matrix{Float64}:
 0.000180437  -0.689047      0.306651      0.656643     4.63813e-5
 0.000374214  -0.68147       0.0341524    -0.731049     9.61922e-5
 0.000663598  -0.246602     -0.951209      0.185441     0.000170579
 0.707107      0.000285257   0.000295826   1.68258e-5  -0.707107
 0.707107      0.000482647   0.000500531   2.8469e-5    0.707107

In [5]:
gradient(f->valuation([1, 1, 1, 1, 1], f=f), rand(5))

5-element Vector{Float64}:
 0.0009414705693620168
 0.004138287329998352
 0.047109437497880824
 0.5048887433368149
 1.3524586802833998